# Generate Summary Datasets
I will average over all the small CSVs with the labeled and unlabeled datasets (e.g. pdc_1.csv from labeled) and generate two summary datasets: summary_labeled.tsv and summary_unlabeled.tsv. 

In [1]:
import numpy as np
import pandas as pd
from clustergrammer_widget import *
net = Network(clustergrammer_widget)